In [30]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(iris_data,columns=iris.feature_names)
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


#### 5개의 컬럼으로 구성되어 있고 null 값은 없으며, 모두 float 형이므로 데이터 전처리 생략

In [32]:
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [39]:
print("아이리스 품종 명:",iris.target_names)

iris_label = iris.target
print("데이터 라벨: ",iris_label)

아이리스 품종 명: ['setosa' 'versicolor' 'virginica']
데이터 라벨:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label, test_size=0.2, \
                                                    random_state=121,stratify=iris_label)

#### 데이터의 80%를 학습을 위해, 20%를 테스트를 위해, stratify=iris_label 을 기준으로 동일한 비율의 라벨을 포함하게 섞어서 나눈다. 

In [47]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dtree = DecisionTreeClassifier()

parameter = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

grid_dtree = GridSearchCV(dtree, param_grid=parameter, cv=3, refit=True)
grid_dtree.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

#### 명시된 하이퍼 파라메터(max_depth, min_samples_split)를 3 Fold 로 테스트 해보고 그중 가장 좋은 하이퍼 파라메터 세트로 셋팅하겠다.

In [49]:
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[["params",'mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.658333,5,0.650,0.675,0.65
1,"{'max_depth': 1, 'min_samples_split': 3}",0.658333,5,0.650,0.675,0.65
2,"{'max_depth': 2, 'min_samples_split': 2}",0.925000,3,0.975,0.950,0.85
3,"{'max_depth': 2, 'min_samples_split': 3}",0.925000,3,0.975,0.950,0.85
4,"{'max_depth': 3, 'min_samples_split': 2}",0.933333,1,1.000,0.950,0.85
5,"{'max_depth': 3, 'min_samples_split': 3}",0.933333,1,1.000,0.950,0.85


#### index 4,5 번이 가장 좋은 파라메터 세트인 것을 확인할 수 있다.

In [50]:
from sklearn.metrics import accuracy_score

estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print("테스트 데이터 세트 정확도:{0:.4f}".format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도:0.9667
